<a href="https://colab.research.google.com/github/TC2008B-Team5/Multiagent-Systems-T5/blob/main/MESA_Team5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>